# ベクトルストア作成用 notebook

- 指定したファイルを Azure AI Foundry プロジェクトへアップロード
- Vector Store を作成し、インデックス処理までブロック
- 完了した Vector Store の ID を出力


In [1]:
import sys
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FilePurpose

In [ ]:
### ファイル存在チェック
file_paths = [
    Path(p).expanduser().resolve()
    for p in [
        "./data/merge_tags_content.txt",
    ]
]
for p in file_paths:
    if not p.exists():
        raise FileNotFoundError(p)

In [6]:
### 1. 環境変数読み子も
load_dotenv()

ENDPOINT = os.getenv("PROJECT_ENDPOINT")
if not ENDPOINT:
    raise EnvironmentError("PROJECT_ENDPOINT is not set in environment variables.")

In [8]:
### 2. クライアントの生成
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=ENDPOINT, credential=credential)
agents_client = AgentsClient(endpoint=ENDPOINT, credential=credential)

In [ ]:
### 3. ファイルアップロード
uploaded_files = []
for path in file_paths:
    print(f"Uploading {path.name} ...")
    u_file = agents_client.files.upload_and_poll(
        file_path=str(path), purpose=FilePurpose.AGENTS
    )
    print(f" Uploaded: {u_file.id}")
    uploaded_files.append(u_file.id)

Uploading merge_tags_content.txt ...
 Uploaded: assistant-BWoD95F1UaKK8vyUkoLXnu


In [ ]:
### 4. Vector Store 作成 & 完了待ち
print("Creating vector store (this may take a few minuts) ...")
vector_store = agents_client.vector_stores.create_and_poll(
    name="vectorstore-" + os.path.splitext(file_paths[0].name)[0],
    file_ids=uploaded_files,
)
print(f"Vector store ready: {vector_store.id}")

Creating vector store (this may take a few minuts) ...
Vector store ready: vs_FtENaUd4hkL9uMBE1wFFQQIY


In [18]:
### 5. 結果出力（標準出力 & JSONファイル）
print("\n=== VECTOR_STORE_ID ===")
print(vector_store.id)

# 伴わせて JSON でも保存しておく
out = {"vector_store_id": vector_store.id}
with open("vector_store.json", "w", encoding="utf-8") as fp:
    json.dump(out, fp, indent=2, ensure_ascii=False)
print("Saved vector_store.json")


=== VECTOR_STORE_ID ===
vs_FtENaUd4hkL9uMBE1wFFQQIY
Saved vector_store.json


# 作成したベクトルストアの利用確認


In [11]:
from azure.ai.agents.models import FileSearchTool, RunStatus

In [24]:
### 6. FileSearchToolを初期化し、agentを作成
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

instruction = """
    あなたは、コロナ対策支援事業について答えるアシスタントです。
    file_searchツールを使い、質問に関連する支援事業を見つけて、対象となる支援事業を紹介してください。
    """

MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")

agent = agents_client.create_agent(
    model=MODEL_DEPLOYMENT_NAME,
    name="rag-agent",
    instructions=instruction,
    tools=file_search.definitions,
    tool_resources=file_search.resources,
)
print("Agent ID:", agent.id)

Agent ID: asst_PPVXRYRlDUeppLFHQ7L2V7L1


In [ ]:
### 7. スレッド+メッセージ->ラン実行
user_msg = (
    "コロナで働けないため、家賃の支払いが厳しいです。家賃の補助支援はないですか？"
)
thread = agents_client.threads.create()
agents_client.messages.create(thread_id=thread.id, role="user", content=user_msg)

run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

if run.status == RunStatus.FAILED:
    print(run.last_error.code, "-", run.last_error.message)
else:
    for msg in agents_client.messages.list(thread_id=thread.id):
        if msg.role != "tool":
            print(f"{msg.role}: {msg.text_messages[0].text.value}")

MessageRole.AGENT: コロナの影響で働けず、家賃の支払いが厳しい場合に利用できる家賃補助について、いくつかの制度があります。

1. 住居確保給付金  
「住居確保給付金」があり、これは離職や収入減少によって住居を失った、または失うおそれのある方を対象に、家賃相当額を有期で給付する制度です。令和2年4月20日からは、コロナにより「離職または廃業に至っていないが同等の状況にある方」も対象になっています。詳細や申請方法は、お住まいの自立相談支援機関で確認ください【4:11†merge_tags_content.txt】【4:5†merge_tags_content.txt】。

2. 生活困窮者自立支援金  
また、「新型コロナウイルス感染症生活困窮者自立支援金」もあり、住居確保給付金と併給できる場合があります。支給額や期間の詳細も併せてご確認ください【4:0†merge_tags_content.txt】。

3. 生活保護・都営住宅等  
さらに困難な場合は、生活保護制度も選択肢となります。生活保護の住宅扶助により家賃補助が受けられます。都営住宅にお住まいの方は、一定基準以下の収入であれば家賃の減額申請も可能です【4:9†merge_tags_content.txt】【4:7†merge_tags_content.txt】。

お住まいの自治体や区役所の相談窓口で、具体的な申請方法や自身が対象に該当するかご相談されることをおすすめします。
MessageRole.USER: コロナで働けないため、家賃の支払いが厳しいです。家賃の補助支援はないですか？
